# 0. Environment setup and dataset preparation

Please follow the instructions in the README file to setup the environment and prepare the dataset.

# 1. Configurations

We povide the default values of the parameters in `configs/default.yml`, with comments explaining the meaning of most parameters. For each individual scene, we can override the default values by providing a separate configuration file in the `configs` directory. For example, the configuration file for the Lego scene from the NeRF Synthetic dataset is `configs/nerfsyn/lego.yml`. 

# 2. Novel view synthesis using PAPR

## 2.1 Training

To synthesize novel views of a scene, we need to train a PAPR model. We provide the configuration files for each scene in the NeRF Synthetic dataset and the Tanks and Temples dataset under the `configs` directory. 

The configuration files specify the training parameters and the dataset paths. Detailed comments are provided in `configs/demo.yml` to help you understand the parameters.

To train a PAPR model, on the lego scene of NeRF Synthetic dataset for example, run the following command:

In [ ]:
!python train.py --opt configs/nerfsyn/lego.yml

You can find the trained model in the `experiments` folder. You can specify the folder name using the `index` parameter in the configuration files. You can also find the training logs and middle plots in the folder during training.

## 2.2 Testing

Once the training is finished, you can synthesize images from the test views by running:

In [ ]:
!python test.py --opt configs/nerfsyn/lego.yml

The generated images and evaluation metrics will be saved under `experiments/{index}/test`.

# 3. Improve the performance for in-the-wild data with exposure control

Whenever people take real life photos with auto-exposure turned on, the exposure of the images can vary significantly across different views, for example the Tanks and Temples dataset. To improve the performance of PAPR on in-the-wild data, we propose a simple yet effective method to control the exposure of the rendered images. We introduce an additional latent code input into our model and finetune it using a technique called [conditional Implicit Maximum Likelihood Estimation (cIMLE)](https://arxiv.org/abs/2004.03590). During test time, we can manipulate the exposure of the rendered image by changing the latent code input.

One can refer to Section 4.4 and Appendix A.8 in the paper for more details.

## 3.1 Finetuning

To finetune the model with exposure control, we need a pretrained model generated by following the instructions in [Section 1.1](##-1.1-Training). We provide a exposure control configuration file for the Caterpillar scene from the Tanks and Temples dataset in `configs/t2/Caterpillar_exposure_control.yml`. You need to specify the path to the pretrained model by setting the `load_path` parameter in the configuration file.

To finetune the model with exposure control, run the following command:



In [ ]:
!python exposure_control_finetune.py --opt configs/t2/Caterpillar_exposure_control.yml

## 3.2 Testing

To render all the test views with a single random exposure level, run:

In [ ]:
!python test.py --opt configs/t2/Caterpillar_exposure_control.yml --exp

## 3.3 Generating images with different exposure levels

You can generate images with different exposure levels by changing the latent code input for a given view. We recommend to increase the `shading_code_scale` to generate images with more diverse exposure levels. You can either change the parameter in the configuration file or specify it in the command line. For example, to generate images with different exposure levels for the Caterpillar scene, run:

In [ ]:
!python test.py --opt configs/t2/Caterpillar_exposure_control.yml --exp --random --scale 8.0 --num_samples 20 --view 0

## 3.4 Generating images with continuous exposure changes

After you generate the images with different exposure levels, you can interpolate the latent codes to generate images with intermediate exposure levels. For example, to generate images with intermediate exposure levels for the Caterpillar scene, run:

In [ ]:
!python test.py --opt configs/t2/Caterpillar_exposure_control.yml --exp --intrp --scale 8 --num_samples 20 --view 0 --start_index 8 --end_index 11 --num_intrp 10

# 4. Use PAPR for your own dataset

## 4.1 Prepare the dataset

You can refer to this [issue](https://github.com/zvict/papr/issues/3#issuecomment-1907260683) for the instructions on how to prepare the dataset.

## 4.2 Configuration

You need to create a new configuration file for your own dataset, and put it under `configs`. The parameter `dataset.type` in the configuration file specifies the type of the dataset. If your dataset is in the same format as the NeRF Synthetic dataset, you can directly set `dataset.type` to `"synthetic"`. Otherwise, you need to implement your own python script to load the dataset under the `dataset` folder, and add it in the function `load_meta_data` in `dataset/utils.py`.

Most default parameters in `configs/default.yml` are general and can be used for your own dataset. You can specify the parameters that are specific to your dataset in the configuration file you created, similar to the configuration files for the NeRF Synthetic dataset and the Tanks and Temples dataset.